In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import warnings
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import torch
import torch.nn as nn

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
# 加载数据集
data = pd.read_csv('data33.csv')
clinicalInput = data


# LSTM

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_validate, cross_val_predict
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization
from tensorflow.keras import layers
from tensorflow.keras import backend as K


In [ ]:
#determine the number of classes(labels)
label_encoder = LabelEncoder().fit(clinicalInput.PATIENT_UPDATE)
labels = label_encoder.transform(clinicalInput.PATIENT_UPDATE)
classes = list(label_encoder.classes_)

#data preparation
clinicalInput = clinicalInput.drop(["PATIENT_UPDATE"],axis=1)

#determine number of features and classes
nb_features = 35
nb_classes = len(classes)

#Standardization of train data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(clinicalInput.values)
clinicalInput = scaler.transform(clinicalInput.values)

#split train data into validation and train
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(clinicalInput, labels, test_size=0.3)

In [ ]:
#split train data into validation and train
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(clinicalInput, labels, test_size=0.3)

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
#determine the categories of labels
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

#resize the input data
X_train = np.array(X_train).reshape(3989,35,1)
X_valid = np.array(X_valid).reshape(1710,35,1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv1D, Dropout, MaxPooling1D, Flatten, LSTM,BatchNormalization

In [ ]:
model = Sequential()
model.add(LSTM(256,input_shape=(nb_features,1)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add((Flatten()))
model.add(Dropout(0.25))
model.add(Dense(2048, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(16, activation="relu"))
model.add(Dense(nb_classes, activation="softmax")) 
model.summary()
    
from tensorflow.keras.optimizers import SGD
opt = SGD(lr=0.3, decay=0.1, momentum=0.2, nesterov=True)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer = opt, metrics=["accuracy"])

In [ ]:
score = model.fit(X_train, y_train,batch_size=100, epochs = 100, validation_data=(X_valid,y_valid))

In [ ]:
#plot training and validation values
import matplotlib.pyplot as plt
plt.plot(score.history["accuracy"])
plt.plot(score.history["val_accuracy"])
plt.title("Model Accuracies")
plt.ylabel("Accuracy")
plt.xlabel("Number of Epochs")
plt.legend(["Train","Test"], loc="upper left")
plt.show()

In [ ]:
plt.plot(score.history["loss"],color="g")
plt.plot(score.history["val_loss"],color="r")
plt.title("Model Losses")
plt.ylabel("Loss")
plt.xlabel("Number of Epochs")
plt.legend(["Train","Test"], loc="upper left")
plt.show()

In [ ]:
y_score = model.predict((X_valid))
y_pred=np.around(y_score[:, 1])
y_test=y_valid[:, 1]

from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
precision, recall, f1 = precision_recall_fscore_support(y_test,y_pred,average='macro')[:-1]
print("accuracy: ", accuracy)
print("precision: ",precision)
print("recall: ",recall)
print("f1: ",f1)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
import seaborn as sns
def print_confusionMatrix(Y_TestLabels, PredictedLabels):
    confusionMatx = confusion_matrix(Y_TestLabels, PredictedLabels)
    
    precision = confusionMatx/confusionMatx.sum(axis = 0)
    
    recall = (confusionMatx.T/confusionMatx.sum(axis = 1)).T
    
    sns.set(font_scale=1.5)
    
    # confusionMatx = [[1, 2],
    #                  [3, 4]]
    # confusionMatx.T = [[1, 3],
    #                   [2, 4]]
    # confusionMatx.sum(axis = 1)  axis=0 corresponds to columns and axis=1 corresponds to rows in two diamensional array
    # confusionMatx.sum(axix =1) = [[3, 7]]
    # (confusionMatx.T)/(confusionMatx.sum(axis=1)) = [[1/3, 3/7]
    #                                                  [2/3, 4/7]]

    # (confusionMatx.T)/(confusionMatx.sum(axis=1)).T = [[1/3, 2/3]
    #                                                    [3/7, 4/7]]
    # sum of row elements = 1
    
    labels = ["0","1"]

    
    plt.figure(figsize=(18,10),dpi=60)
    sns.heatmap(confusionMatx, cmap = "YlGnBu_r", annot = True, fmt = ".1f", xticklabels=labels, yticklabels=labels)
    plt.title("LSTM Confusion Matrix", fontsize = 30)
    plt.xlabel('Predicted Class', fontsize = 30)
    plt.ylabel('Original Class', fontsize = 30)
    plt.tick_params(labelsize = 25)
    plt.xticks(rotation = 90)
    plt.savefig("Pictures/LSTM-Confusion Matrix.pdf")
    plt.savefig("Pictures/LSTM-Confusion Matrix.png")
    plt.show()

    
    print("-"*125)
    
    plt.figure(figsize=(18,10),dpi=60)
    sns.heatmap(precision, cmap = "YlGnBu_r", annot = True, fmt = ".2f", xticklabels=labels, yticklabels=labels)
    plt.title("LSTM Precision Matrix", fontsize = 30)
    plt.xlabel('Predicted Class', fontsize = 30)
    plt.ylabel('Original Class', fontsize = 30)
    plt.tick_params(labelsize = 25)
    plt.xticks(rotation = 90)
    plt.savefig("Pictures/LSTM-Precision Matrix.pdf")
    plt.savefig("Pictures/LSTM-Precision Matrix.png")
    plt.show()
    print("-"*125)
    
    plt.figure(figsize=(18,10),dpi=60)
    sns.heatmap(recall, cmap = "YlGnBu_r", annot = True, fmt = ".2f", xticklabels=labels, yticklabels=labels)
    plt.title("LSTM Recall Matrix", fontsize = 30)
    plt.xlabel('Predicted Class', fontsize = 30)
    plt.ylabel('Original Class', fontsize = 30)
    plt.tick_params(labelsize = 25)
    plt.xticks(rotation = 90)
    plt.savefig("Pictures/LSTM-Recall Matrix.pdf")
    plt.savefig("Pictures/LSTM-Recall Matrix.png")
    plt.show()
    sensitivity = confusionMatx[0,0]/(confusionMatx[0,0]+confusionMatx[1,0])
    print('Sensitivity : ', sensitivity )

    specificity = confusionMatx[1,1]/(confusionMatx[1,1]+confusionMatx[0,1])
    print('Specificity : ', specificity)
    
def plot_roc_curve(fpr, tpr):
    plt.figure()
    lw = 2
    roc_auc = auc(fpr,tpr)
    plt.plot(fpr, tpr, color='darkorange',lw=lw, 
             label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0,1], [0,1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0, 1.0])
    plt.ylim([0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    #plt.title('')
    plt.legend(loc="lower right")
    plt.show()  

In [ ]:
print_confusionMatrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(y_test, y_score[:,1])
plot_roc_curve(fpr, tpr)    

In [ ]:
df1 = pd.DataFrame({'y_test':y_test,'y_score[:,1]': y_score[:,1]})
df1.to_csv('Pictures/pre_lab_4.csv',index=False)